# Generating recommendations

In this notebook, I will cover how to use trained models to generate recommendations. We will explore three methods to do so:

- Model score
- Global feature importance
- Local feature importance

First, we load data and the best model we've trained, the one from the third model [notebook](https://github.com/hundredblocks/ml-powered-applications/blob/master/notebooks/third_model.ipynb).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.externals import joblib
from lime.lime_tabular import LimeTabularExplainer
import sys
sys.path.append("..")
import warnings
warnings.filterwarnings('ignore')

from ml_editor.data_processing import (
    format_raw_df, 
    get_split_by_author,
    get_vectorized_series, 
    get_feature_vector_and_label
)
from ml_editor.model_evaluation import get_feature_importance
from ml_editor.model_v3 import get_question_score_from_input, get_features_from_input_text
from ml_editor.explanation_generation import FEATURE_DISPLAY_NAMES
from ml_editor.model_v2 import POS_NAMES

data_path = Path('../data/writers_with_features.csv')
df = pd.read_csv(data_path)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/emmanuel.ameisen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/emmanuel.ameisen/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
clf = joblib.load(Path("../models/model_3.pkl")) 

In [3]:
train_df, test_df = get_split_by_author(df, test_size=0.2, random_state=40)

In [4]:
features = ["num_questions", 
               "num_periods",
               "num_commas",
               "num_exclam",
               "num_quotes",
               "num_colon",
               "num_stops",
               "num_semicolon",
               "num_words",
               "num_chars",
               "num_diff_words",
               "avg_word_len",
               "polarity"
              ]
features.extend(POS_NAMES.keys())

y_test = test_df["Score"] > test_df["Score"].median()
X_test = test_df[features].astype(float)

## Using feature values

The simplest method does not even involve a model. We can look at aggregate feature values for both high and low score questions, and pick the features with the largest difference as ones to recommend to users. Below, I show an example of how to do just that in a fea lines of pandas.

In [5]:
features_and_labels = df.copy()
features_and_labels["label"] = features_and_labels["Score"] > features_and_labels["Score"].median()
features_and_labels = features_and_labels[["label"] + features].copy()

In [6]:
class_feature_values = features_and_labels.groupby("label").mean()
class_feature_values = class_feature_values.round(3)
class_feature_values = class_feature_values.transpose()
class_feature_values.rename(columns={
    False: "low score",
    True: "high score"
}, inplace=True)
class_feature_values["relative difference"] = abs(class_feature_values["high score"] - class_feature_values["low score"]) / class_feature_values["high score"]
class_feature_values.sort_values(by="relative difference", inplace=True, ascending=False)
class_feature_values

label           low score  high score  relative difference
SYM                 0.001       0.000                  inf
num_exclam          0.019       0.015             0.266667
PROPN               0.005       0.004             0.250000
num_colon           0.094       0.081             0.160494
num_chars         822.104     967.032             0.149869
num_diff_words    177.428     206.859             0.142276
num_quotes          0.216       0.199             0.085427
num_periods         0.814       0.754             0.079576
num_commas          0.673       0.728             0.075549
ADV                 0.013       0.014             0.071429
num_semicolon       0.013       0.014             0.071429
ADJ                 0.013       0.014             0.071429
num_questions       0.432       0.409             0.056235
DET                 0.024       0.023             0.043478
NOUN                0.038       0.037             0.027027
polarity            0.099       0.100             0.010000
num_words          21.638      21.480             0.007356
num_stops          10.537      10.610             0.006880
avg_word_len        3.849       3.874             0.006453
ADP                 0.021       0.021             0.000000
NUM                 0.002       0.002             0.000000
PART                0.006       0.006             0.000000
PRON                0.015       0.015             0.000000
PUNCT               0.025       0.025             0.000000
VERB                0.040       0.040             0.000000
AUX                 0.000       0.000                  NaN
CONJ                0.000       0.000                  NaN
INTJ                0.000       0.000                  NaN
SCONJ               0.000       0.000                  NaN
X                   0.000       0.000                  NaN

## Using feature importance

Looking at feature values is a good start, but we can do better. Since we've trained a model, we can leverage its feature importances to recommends aspects to improve.

In [7]:
k = 5

print("Using %s features in total" % len(features))
print("Top %s importances:\n" % k)
print('\n'.join(["%s: %.2g" % (tup[0], tup[1]) for tup in get_feature_importance(clf, np.array(features))[:k]]))

print("\nBottom %s importances:\n" % k)
print('\n'.join(["%s: %.2g" % (tup[0], tup[1]) for tup in get_feature_importance(clf, np.array(features))[-k:]]))

Using 30 features in total
Top 5 importances:

num_chars: 0.053
num_periods: 0.052
num_questions: 0.051
num_diff_words: 0.049
ADJ: 0.048

Bottom 5 importances:

X: 0.011
num_semicolon: 0.0077
num_exclam: 0.0072
CONJ: 0
SCONJ: 0


## Using model score

In addition to feature importance, we can leverage a model's output score to give a user a current score representing the quality of their question.

In [8]:
example_question = """
Is displaying a binary class enough to guide a user
"""
q_score = get_question_score_from_input(example_question)

print("%s probability of being a good question" % q_score)

100%|██████████| 1/1 [00:00<00:00, 846.14it/s]

0.46 probability of being a good question


## Local recommendations with LIME

In the approaches above, recommendations are based on general information and only the score is specific to a specific input question. We'd like to provide recommendations tailored to the inputs our users give us. We'll see how to do that using LIME.

### On examples from test data

First, I'll show how to do this on examples from the held out test set.

In [9]:
explainer = LimeTabularExplainer(
    train_df[features].values,
    feature_names=features,
    class_names=["low", "high"],
)

In [10]:
idx = 8
exp = explainer.explain_instance(
    test_df[features].iloc[idx, :],
    clf.predict_proba,
    num_features=10,
    labels=(1,),
)
print("Q: %s" % test_df["full_text"].iloc[idx])
exp.show_in_notebook(show_table=True, show_all=False)

Q: Where can subject matter experts volunteer to work on proofing the technical details in fiction? I am not a writer (yet!), but I have been working in the computer field for over 10 years and would love to put in some time volunteering my services as a subject matter expert (SME) for programming and computer related story components.  Is there a central place that such SMEs can peddle their knowledge?  Is anyone in need here?



In [11]:
exp.as_list()

[('num_diff_words <= 88.00', -0.035447632047062),
 ('INTJ <= 0.00', 0.022438295339021364),
 ('num_questions > 0.57', 0.02017873972197071),
 ('ADJ <= 0.01', -0.019708870432852417),
 ('num_periods <= 0.50', 0.019007352241933527),
 ('num_commas <= 0.39', -0.014478245193243627),
 ('num_colon <= 0.00', 0.014162491875899515),
 ('num_quotes <= 0.00', 0.009834607843294424),
 ('CONJ <= 0.00', 0.0),
 ('SCONJ <= 0.00', 0.0)]

For a given question, the list of explanations can serve as a set of recommendations.

### On new examples

Let's now extend this method to show explanations for new examples.

In [12]:
feats = get_features_from_input_text(example_question)

exp = explainer.explain_instance(
    feats,
    clf.predict_proba,
    num_features=10,
    labels=(1,),
)
print("Q: %s" % example_question)
exp.show_in_notebook(show_table=True, show_all=False)

100%|██████████| 1/1 [00:00<00:00, 693.39it/s]


Q: 
Is displaying a binary class enough to guide a user



In [13]:
exp.as_list()

[('num_chars <= 408.00', -0.04638125008968274),
 ('num_diff_words <= 88.00', -0.04001736657685673),
 ('num_questions <= 0.19', -0.034455245693925676),
 ('INTJ <= 0.00', 0.020453263851357692),
 ('num_periods <= 0.50', 0.019987802473847377),
 ('DET > 0.03', -0.017969976505925366),
 ('PROPN <= 0.00', 0.01203189521428601),
 ('num_colon <= 0.00', 0.012010208985765027),
 ('CONJ <= 0.00', 0.0),
 ('SCONJ <= 0.00', 0.0)]

Below, I'll write a few functions to display the recommendations in a clearer way to users.

In [14]:
def parse_explanations(exp_list):
    global FEATURE_DISPLAY_NAMES
    parsed_exps = []
    for feat_bound, impact in exp_list:
        conditions = feat_bound.split(" ")

        # We ignore doubly bounded conditions , e.g. 1 <= a < 3 because
        # they are harder to formulate as a recommendation
        if len(conditions) == 3:
            feat_name, order, threshold = conditions

            simple_order = simplify_order_sign(order)
            recommended_mod = get_recommended_modification(simple_order, impact)

            parsed_exps.append(
                {
                    "feature": feat_name,
                    "feature_display_name": FEATURE_DISPLAY_NAMES[feat_name],
                    "order": simple_order,
                    "threshold": threshold,
                    "impact": impact,
                    "recommendation": recommended_mod,
                }
            )
    return parsed_exps

def get_recommended_modification(simple_order, impact):
    bigger_than_threshold = simple_order == ">"
    has_positive_impact = impact > 0

    if bigger_than_threshold and has_positive_impact:
        return "No need to decrease"
    if not bigger_than_threshold and not has_positive_impact:
        return "Increase"
    if bigger_than_threshold and not has_positive_impact:
        return "Decrease"
    if not bigger_than_threshold and has_positive_impact:
        return "No need to increase"

def simplify_order_sign(order_sign):
    if order_sign in ["<=", "<"]:
        return "<"
    if order_sign in [">=", ">"]:
        return ">"
    return order_sign
    
parsed_exps = parse_explanations(exp.as_list())        
parsed_exps

[{'feature': 'num_chars',
  'feature_display_name': 'word count',
  'order': '<',
  'threshold': '408.00',
  'impact': -0.04638125008968274,
  'recommendation': 'Increase'},
 {'feature': 'num_diff_words',
  'feature_display_name': 'vocabulary diversity',
  'order': '<',
  'threshold': '88.00',
  'impact': -0.04001736657685673,
  'recommendation': 'Increase'},
 {'feature': 'num_questions',
  'feature_display_name': 'frequency of question marks',
  'order': '<',
  'threshold': '0.19',
  'impact': -0.034455245693925676,
  'recommendation': 'Increase'},
 {'feature': 'INTJ',
  'feature_display_name': 'frequency of interjections',
  'order': '<',
  'threshold': '0.00',
  'impact': 0.020453263851357692,
  'recommendation': 'No need to increase'},
 {'feature': 'num_periods',
  'feature_display_name': 'frequency of periods',
  'order': '<',
  'threshold': '0.50',
  'impact': 0.019987802473847377,
  'recommendation': 'No need to increase'},
 {'feature': 'DET',
  'feature_display_name': 'frequenc

In [15]:
def get_recommendation_string_from_parsed_exps(exp_list):
    recommendations = []
    for feature_exp in exp_list:
        recommendation = "%s %s" % (
            feature_exp["recommendation"],
            feature_exp["feature_display_name"],
        )
        recommendations.append(recommendation)
    return recommendations

get_recommendation_string_from_parsed_exps(parsed_exps)

['Increase word count',
 'Increase vocabulary diversity',
 'Increase frequency of question marks',
 'No need to increase frequency of interjections',
 'No need to increase frequency of periods',
 'Decrease frequency of determiners',
 'No need to increase frequency of proper nouns',
 'No need to increase frequency of colons',
 'Increase frequency of coordinating conjunctions',
 'Increase frequency of subordinating conjunctions']

The suggestions above are much more understandable, and could be displayed to users looking for feedback. Let's see an example below.

In [16]:
def get_recommendation_and_prediction_from_text(input_text, num_feats=10):
    global clf, explainer
    feats = get_features_from_input_text(input_text)

    pos_score = clf.predict_proba([feats])[0][1]
    exp = explainer.explain_instance(
        feats, clf.predict_proba, num_features=num_feats, labels=(1,)
    )
    parsed_exps = parse_explanations(exp.as_list())
    recs = get_recommendation_string_from_parsed_exps(parsed_exps)
    return recs, pos_score


recos, score = get_recommendation_and_prediction_from_text(example_question)
print("%s score" % score)
print(*recos, sep="\n")

100%|██████████| 1/1 [00:00<00:00, 731.48it/s]


0.46 score
Increase word count
Increase vocabulary diversity
Increase frequency of question marks
No need to increase frequency of interjections
Decrease frequency of determiners
No need to increase frequency of periods
Decrease word count
No need to increase frequency of colons
Increase frequency of subordinating conjunctions
Increase frequency of coordinating conjunctions


## Following successive recommendations

In order to test our proposed workflow. Let's input a question and follow our model's recommendation a few times to see if it (and its score) improves.

In [17]:
recos, score = get_recommendation_and_prediction_from_text(
    """
I want to learn how models are made
"""
)
print("%s score" % score)
print(*recos, sep="\n")

100%|██████████| 1/1 [00:00<00:00, 659.69it/s]


0.4 score
Increase word count
Increase vocabulary diversity
Increase frequency of question marks
No need to increase frequency of periods
No need to increase frequency of interjections
Increase frequency of adjectives
No need to increase frequency of determiners
Decrease word count
Increase frequency of commas
Increase frequency of subordinating conjunctions


Let's increase the length of text, the number of questions, and diversify the vocabulary.

In [18]:
recos, score = get_recommendation_and_prediction_from_text(
    """
I'd like to learn about building machine learning products. 
Are there any good product focused resources? 
Would you be able to recommend educational books?
"""
)

print("%s score" % score)
print(*recos, sep="\n")

100%|██████████| 1/1 [00:00<00:00, 616.45it/s]


0.46 score
Increase word count
Increase vocabulary diversity
No need to decrease frequency of question marks
Increase frequency of adverbs
No need to increase frequency of interjections
No need to increase frequency of proper nouns
No need to decrease frequency of adjectives
Increase frequency of coordinating conjunctions
Increase frequency of subordinating conjunctions


Great, the score increased, let's follow these recommendations once more and increase the length of our text, and the variety of our vocabulary, while keeping the number of question marks constant.

In [19]:
recos, score = get_recommendation_and_prediction_from_text(
    """
I'd like to learn more about ML, specifically how to build ML products.
When I attempt to build such products, I always face the same challenge: 
how do you go beyond a model?
What are the best practices to use a model in a concrete application?
Are there any good product focused resources? 
Would you be able to recommend educational books?
"""
)

print("%s score" % score)
print(*recos, sep="\n")

100%|██████████| 1/1 [00:00<00:00, 285.39it/s]


0.52 score
Increase word count
Increase vocabulary diversity
No need to decrease frequency of question marks
No need to increase frequency of interjections
Decrease frequency of colons
No need to decrease frequency of adjectives
No need to increase frequency of determiners
No need to increase frequency of periods
Increase frequency of coordinating conjunctions
Increase frequency of subordinating conjunctions


We now have a machine learning powered editing assistant! There are **many** ways to improve it, such as iterating on classification performance or building features that are easier to understand for the user. To make such improvements, follow the recommendations in part three of tthe book: Building ML Powered Applications.